<a href="https://colab.research.google.com/github/BotaoJin/Code-for-Thesis/blob/main/AD_EnKF_nonlinear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
!pip install torchdiffeq
from torchdiffeq import odeint
from torchdiffeq import odeint_adjoint
import numpy as np
import matplotlib.pyplot as plt
from torch.distributions.multivariate_normal import MultivariateNormal

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
class Lorenz63(nn.Module):
  def __init__(self, sigma, beta, rho, x_dim = 3):
      super().__init__()
      self.sigma = nn.Parameter(sigma)
      self.beta = nn.Parameter(beta)
      self.rho = nn.Parameter(rho)
      self.x_dim = 3

  def forward(self, t, u):
    sigma = self.sigma
    beta = self.beta
    rho = self.rho
    out = torch.stack((sigma*(u[...,1]-u[...,0]), rho*u[..., 0]-u[..., 0]*u[..., 2]-u[..., 1], u[..., 0]*u[..., 1]-beta*u[..., 2]), dim = -1)
    return out

In [ ]:
x_dim = 3
y_dim = 3
H = torch.eye(3)
cov = torch.eye(3)
gamma = .1
T_ = 100
t1 = torch.tensor([0., .01])

def generate_data(ode_func):
  x0 = torch.tensor([1., 1., 1.])
  X = torch.empty(T_+1, x_dim)
  Y = torch.empty(T_, y_dim)
  X[0] = x0

  for t in range(T_):
    X[t+1] = odeint(ode_func, X[t], t1)[-1]
    X[t+1] = MultivariateNormal(X[t+1], gamma**2*cov).sample()
    Y[t] = MultivariateNormal(X[t+1], gamma**2*cov).sample()

  return X, Y

In [ ]:
# generate data
true_sigma = torch.tensor(10.0)
true_beta = torch.tensor(8.0/3.0)
true_rho = torch.tensor(28.0)

true_ode_func = Lorenz63(sigma = true_sigma, beta = true_beta, rho = true_rho)
X, Y = generate_data(true_ode_func)

In [ ]:
def EnKF(ode_func, Y, N_ensem, t_span, x0 = torch.tensor([1., 1., 1.])):
  T = Y.shape[-2]
  X = x0.expand((N_ensem, x_dim))
  res = torch.empty(T+1, N_ensem, x_dim)
  res[0] = X
  mean = torch.zeros(x_dim)
  log_likelihood = torch.tensor(0.)

  for t in range(T):
    # Forcast Step
    X = odeint_adjoint(ode_func, X, t_span, method = 'rk4', adjoint_method = 'rk4')[-1]
    X = X + MultivariateNormal(mean.expand((N_ensem, x_dim)), (gamma**2)*cov).sample() # model error for X: dim = (N_ensem, x_dim)
    X_m = X.mean(dim = -2).unsqueeze(-2) # dim = (1, x_dim)
    X_ct = X - X_m

    # Analysis Step
    y_obs_j = Y[t].unsqueeze(-2) # dim = (1, y_dim)
    y_obs_perturb = MultivariateNormal(y_obs_j.expand(N_ensem, y_dim), (gamma**2)*cov).sample()

    C_uu = 1/(N_ensem - 1)*X_ct.transpose(-1, -2)@X_ct # dim = (1, x_dim)
    # In this model, setting H = I
    HX = X
    HX_m = X_m
    HC = C_uu
    HCH_T = HC
    HCH_TR_chol = torch.linalg.cholesky(HCH_T + (gamma**2)*cov)
    d = MultivariateNormal(HX_m.squeeze(-2), scale_tril = HCH_TR_chol)
    log_likelihood += d.log_prob(y_obs_j.squeeze(-2))

    # Update and store X_j^{1:N}
    pre = (y_obs_perturb-HX)@torch.cholesky_inverse(HCH_TR_chol)
    X = X + pre@HC
    res[t+1] = X

  return X, res, log_likelihood

In [ ]:
# gradient ascent (sigma unknown, N = 10)
sigma, beta, rho = torch.tensor(.01), true_beta, true_rho
iter_sigma_GD = [sigma]
iter_grad_sigma = []
eta_sigma = 1e-3

for j in range(200):
  sigma1 = sigma.clone().detach().requires_grad_(True)

  ode_func = Lorenz63(sigma1, beta, rho)
  x_bar, res, loglike = EnKF(ode_func, Y, N_ensem = 10, t_span = t1)
  loglike.backward()
  grad_sigma = ode_func.sigma.grad
  sigma = sigma + eta_sigma*grad_sigma

  iter_sigma_GD.append(sigma)
  iter_grad_sigma.append(grad_sigma)

In [ ]:
# gradient ascent (sigma unknown, N = 100)
sigma, beta, rho = torch.tensor(.01), true_beta, true_rho
iter_sigma_GD1 = [sigma]
iter_grad_sigma1 = []
eta_sigma = 1e-3

for j in range(200):
  sigma1 = sigma.clone().detach().requires_grad_(True)

  ode_func = Lorenz63(sigma1, beta, rho)
  x_bar, res, loglike = EnKF(ode_func, Y, N_ensem = 100, t_span = t1)
  loglike.backward()
  grad_sigma = ode_func.sigma.grad
  sigma = sigma + eta_sigma*grad_sigma

  iter_sigma_GD1.append(sigma)
  iter_grad_sigma1.append(grad_sigma)

In [ ]:
# gradient ascent (sigma unknown, N = 1000)
sigma, beta, rho = torch.tensor(.01), true_beta, true_rho
iter_sigma_GD2 = [sigma]
iter_grad_sigma2 = []
eta_sigma = 1e-3

for j in range(200):
  sigma1 = sigma.clone().detach().requires_grad_(True)

  ode_func = Lorenz63(sigma1, beta, rho)
  x_bar, res, loglike = EnKF(ode_func, Y, N_ensem = 1000, t_span = t1)
  loglike.backward()
  grad_sigma = ode_func.sigma.grad
  sigma = sigma + eta_sigma*grad_sigma

  iter_sigma_GD2.append(sigma)
  iter_grad_sigma2.append(grad_sigma)

In [ ]:
# gradient ascent (beta unknown, N = 10)
sigma, beta, rho = true_sigma, torch.tensor(0.), true_rho
iter_beta_GD = [beta]
iter_grad_beta = []
eta_beta = 1e-4

for j in range(200):
  beta1 = beta.clone().detach().requires_grad_(True)

  ode_func = Lorenz63(sigma, beta1, rho)
  x_bar, res, loglike = EnKF(ode_func, Y, N_ensem = 10, t_span = t1)
  loglike.backward()
  grad_beta = ode_func.beta.grad
  beta = beta + eta_beta*grad_beta

  iter_beta_GD.append(beta)
  iter_grad_beta.append(grad_beta)

In [ ]:
# gradient ascent (beta unknown, N = 100)
sigma, beta, rho = true_sigma, torch.tensor(0.), true_rho
iter_beta_GD1 = [beta]
iter_grad_beta1 = []
eta_beta = 1e-4

for j in range(200):
  beta1 = beta.clone().detach().requires_grad_(True)

  ode_func = Lorenz63(sigma, beta1, rho)
  x_bar, res, loglike = EnKF(ode_func, Y, N_ensem = 100, t_span = t1)
  loglike.backward()
  grad_beta = ode_func.beta.grad
  beta = beta + eta_beta*grad_beta

  iter_beta_GD1.append(beta)
  iter_grad_beta1.append(grad_beta)

In [ ]:
# gradient ascent (beta unknown, N = 1000)
sigma, beta, rho = true_sigma, torch.tensor(0.), true_rho
iter_beta_GD2 = [beta]
iter_grad_beta2 = []
eta_beta = 1e-4

for j in range(200):
  beta1 = beta.clone().detach().requires_grad_(True)

  ode_func = Lorenz63(sigma, beta1, rho)
  x_bar, res, loglike = EnKF(ode_func, Y, N_ensem = 1000, t_span = t1)
  loglike.backward()
  grad_beta = ode_func.beta.grad
  beta = beta + eta_beta*grad_beta

  iter_beta_GD2.append(beta)
  iter_grad_beta2.append(grad_beta)

In [ ]:
# gradient ascent (rho unknown, N = 10)
sigma, beta, rho = true_sigma, true_beta, torch.tensor(0.)
iter_rho_GD = [rho]
iter_grad_rho = []
eta_rho = 1e-3

for j in range(200):
  rho1 = rho.clone().detach().requires_grad_(True)

  ode_func = Lorenz63(sigma, beta, rho1)
  x_bar, res, loglike = EnKF(ode_func, Y, N_ensem = 10, t_span = t1)
  loglike.backward()
  grad_rho = ode_func.rho.grad
  rho = rho + eta_rho*grad_rho

  iter_rho_GD.append(rho)
  iter_grad_rho.append(grad_rho)

In [ ]:
# gradient ascent (rho unknown, N = 100)
sigma, beta, rho = true_sigma, true_beta, torch.tensor(0.)
iter_rho_GD1 = [rho]
iter_grad_rho1 = []
eta_rho = 1e-3

for j in range(200):
  rho1 = rho.clone().detach().requires_grad_(True)

  ode_func = Lorenz63(sigma, beta, rho1)
  x_bar, res, loglike = EnKF(ode_func, Y, N_ensem = 100, t_span = t1)
  loglike.backward()
  grad_rho = ode_func.rho.grad
  rho = rho + eta_rho*grad_rho

  iter_rho_GD1.append(rho)
  iter_grad_rho1.append(grad_rho)

In [ ]:
# gradient ascent (rho unknown, N = 1000)
sigma, beta, rho = true_sigma, true_beta, torch.tensor(0.)
iter_rho_GD2 = [rho]
iter_grad_rho2 = []
eta_rho = 1e-3

for j in range(200):
  rho1 = rho.clone().detach().requires_grad_(True)

  ode_func = Lorenz63(sigma, beta, rho1)
  x_bar, res, loglike = EnKF(ode_func, Y, N_ensem = 1000, t_span = t1)
  loglike.backward()
  grad_rho = ode_func.rho.grad
  rho = rho + eta_rho*grad_rho

  iter_rho_GD2.append(rho)
  iter_grad_rho2.append(grad_rho)

In [ ]:
fig = plt.figure(figsize=(18, 4.5))

plt.subplot(1,3,1)
plt.plot(range(len(iter_sigma_GD)), torch.tensor(iter_sigma_GD).detach().clone(), label = 'N = 10')
plt.plot(range(len(iter_sigma_GD1)), torch.tensor(iter_sigma_GD1).detach().clone(), label = 'N = 100')
plt.plot(range(len(iter_sigma_GD2)), torch.tensor(iter_sigma_GD2).detach().clone(), label = 'N = 1000')
plt.plot(range(len(iter_sigma_GD)), true_sigma * np.ones((len(iter_sigma_GD),)), label = 'True sigma')
plt.xlabel('iterations')
plt.ylabel('values')
plt.legend()

plt.subplot(1,3,2)
plt.plot(range(len(iter_beta_GD)), torch.tensor(iter_beta_GD).detach().clone(), label = 'N = 10')
plt.plot(range(len(iter_beta_GD1)), torch.tensor(iter_beta_GD1).detach().clone(), label = 'N= 100')
plt.plot(range(len(iter_beta_GD2)), torch.tensor(iter_beta_GD2).detach().clone(), label = 'N= 1000')
plt.plot(range(len(iter_beta_GD)), true_beta * np.ones((len(iter_beta_GD),)), label = 'True beta')
plt.xlabel('iterations')
plt.ylabel('values')
plt.legend()

plt.subplot(1,3,3)
plt.plot(range(len(iter_rho_GD)), torch.tensor(iter_rho_GD).detach().clone(), label = 'N = 10')
plt.plot(range(len(iter_rho_GD1)), torch.tensor(iter_rho_GD1).detach().clone(), label = 'N = 100')
plt.plot(range(len(iter_rho_GD2)), torch.tensor(iter_rho_GD2).detach().clone(), label = 'N = 1000')
plt.plot(range(len(iter_rho_GD)), true_rho * np.ones((len(iter_rho_GD),)), label = 'True rho')
plt.xlabel('iterations')
plt.ylabel('values')
plt.legend()


plt.show()

In [ ]:
fig = plt.figure(figsize=(18, 4.5))

plt.subplot(1,3,1)
plt.plot(range(len(iter_grad_sigma)), torch.tensor(iter_grad_sigma).detach().clone(), label = 'grad of sigma with N = 10')
plt.plot(range(len(iter_grad_sigma1)), torch.tensor(iter_grad_sigma1).detach().clone(), label = 'grad of sigma with N = 100')
plt.plot(range(len(iter_grad_sigma2)), torch.tensor(iter_grad_sigma2).detach().clone(), label = 'grad of sigma with N = 1000')
plt.xlabel('iterations')
plt.ylabel('grad log-likelihood')
plt.legend()


plt.subplot(1,3,2)
plt.plot(range(len(iter_grad_beta)), torch.tensor(iter_grad_beta).detach().clone(), label = 'grad of beta with N = 10')
plt.plot(range(len(iter_grad_beta1)), torch.tensor(iter_grad_beta1).detach().clone(), label = 'grad of beta with N = 100')
plt.plot(range(len(iter_grad_beta2)), torch.tensor(iter_grad_beta2).detach().clone(), label = 'grad of beta with N = 1000')
plt.xlabel('iterations')
plt.ylabel('grad log-likelihood')
plt.legend()

plt.subplot(1,3,3)
plt.plot(range(len(iter_grad_rho)), torch.tensor(iter_grad_rho).detach().clone(), label = 'grad of rho with N = 10')
plt.plot(range(len(iter_grad_rho1)), torch.tensor(iter_grad_rho1).detach().clone(), label = 'grad of rho with N = 100')
plt.plot(range(len(iter_grad_rho2)), torch.tensor(iter_grad_rho2).detach().clone(), label = 'grad of rho with N = 1000')
plt.xlabel('iterations')
plt.ylabel('grad log-likelihood')
plt.legend()

plt.show()